# GenePT cell classification performance on the Tabula Sapiens data set

This notebook downloads (if necessary) the Tabula Sapiens data set (https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5)
uses a GenePT-style embedding to embed the cells and then tests classification performance.  This is a benchmark dataset, but we are using it for
both training and testing. We are doing this to "benchmark" our GenePT embeddings on a large dataset with a high number of cell types.


In [2]:
%run notebook_setup.ipynb

autoreload enabled
repo_dir set to /Users/rj/personal/GenePT-tools
File already exists at /Users/rj/personal/GenePT-tools/data/GenePT_emebdding_v2.zip
Extracting files...
Extracting GenePT_emebdding_v2/
Skipping GenePT_emebdding_v2/NCBI_UniProt_summary_of_genes.json - already exists with same size
Skipping GenePT_emebdding_v2/GenePT_gene_embedding_ada_text.pickle - already exists with same size
Skipping GenePT_emebdding_v2/GenePT_gene_protein_embedding_model_3_text.pickle. - already exists with same size
Skipping GenePT_emebdding_v2/NCBI_summary_of_genes.json - already exists with same size
Extraction complete!
Skipping embedding_original_ada_text.parquet - already exists
Skipping embedding_original_large_3.parquet - already exists
Skipping embedding_associations_age_cell_type_drugs_pathways_openai_large.parquet - already exists
Skipping embedding_associations_age_drugs_pathways_openai_large.parquet - already exists
Skipping embedding_associations_cell_type_openai_large.parquet - alrea

## Download the Tabula Sapiens data set

We use the name `1m_cells.h5ad` for this dataset, which is not the best name.

In [3]:
import requests

dataset = "https://datasets.cellxgene.cziscience.com/10df7690-6d10-4029-a47e-0f071bb2df83.h5ad"
# dataset_id = "10df7690-6d10-4029-a47e-0f071bb2df83"

file_path = data_dir / "1m_cells.h5ad"  # adjust this path as needed

if not file_path.exists():
    response = requests.get(dataset, stream=True)
    with open(file_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:  # filter out keep-alive chunks
                file.write(chunk)

In [4]:
import os
# Print the file size of the dataset
print(f"Dataset file size: {os.path.getsize(file_path) / 1e9:.2f} GB")

import h5py
with h5py.File(file_path, 'r') as f:
    # Print the main groups in the HDF5 file
    print("\nMain HDF5 groups:", f.keys())
    
    # Print the type of the expression matrix storage
    print("\nExpression matrix (X) storage type:", type(f['X']))
    
    # Print the components of the sparse matrix storage
    print("\nSparse matrix components:", f['X'].keys())
    
    # Print the chunk size used for data storage
    print("\nChunk size for data storage:", f['X/data'].chunks)
    
    # Check what additional data matrices are stored
    print("\nStored embeddings (e.g. PCA, UMAP):", 
          f['obsm'].keys() if 'obsm' in f else "No embeddings found")
    print("Pairwise relationships (e.g. distances, connectivities):", 
          f['obsp'].keys() if 'obsp' in f else "No pairwise relationships found")
    print("Additional expression matrices (e.g. normalized, raw):", 
          f['layers'].keys() if 'layers' in f else "No additional layers found")

Dataset file size: 45.14 GB

Main HDF5 groups: <KeysViewHDF5 ['X', 'layers', 'obs', 'obsm', 'obsp', 'raw', 'uns', 'var', 'varm', 'varp']>

Expression matrix (X) storage type: <class 'h5py._hl.group.Group'>

Sparse matrix components: <KeysViewHDF5 ['data', 'indices', 'indptr']>

Chunk size for data storage: (53979,)

Stored embeddings (e.g. PCA, UMAP): <KeysViewHDF5 ['X_pca', 'X_scvi', 'X_umap', 'X_umap_scvi_full_donorassay', 'X_uncorrected_alltissues_umap', 'X_uncorrected_umap']>
Pairwise relationships (e.g. distances, connectivities): <KeysViewHDF5 ['connectivities', 'distances']>
Additional expression matrices (e.g. normalized, raw): <KeysViewHDF5 ['decontXcounts', 'scale_data']>


In [5]:
import h5py

with h5py.File(file_path, "r") as f:
    # Look at the structure of the X group
    print("Contents of X group:", list(f["X"].keys()))

    # Look at obs and var to get dimensions
    print("\nContents of obs group:", list(f["obs"].keys()))
    print("Contents of var group:", list(f["var"].keys()))

    # If X contains a sparse matrix, it likely has 'data', 'indices', and 'indptr'
    if "data" in f["X"]:
        print("\nShape of X/data:", f["X"]["data"].shape)
        print("Shape of X/indices:", f["X"]["indices"].shape)
        print("Shape of X/indptr:", f["X"]["indptr"].shape)

Contents of X group: ['data', 'indices', 'indptr']

Contents of obs group: ['10X_run', '_index', '_scvi_batch', '_scvi_labels', 'ambient_removal', 'anatomical_position', 'assay', 'assay_ontology_term_id', 'broad_cell_class', 'cdna_plate', 'cdna_well', 'cell_type', 'cell_type_ontology_term_id', 'compartment', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_assay', 'donor_id', 'donor_method', 'donor_tissue', 'donor_tissue_assay', 'ethnicity_original', 'free_annotation', 'is_primary_data', 'library_plate', 'manually_annotated', 'method', 'n_genes_by_counts', 'notes', 'observation_joinid', 'organism', 'organism_ontology_term_id', 'pct_counts_ercc', 'pct_counts_mt', 'published_2022', 'replicate', 'sample_id', 'sample_number', 'scvi_leiden_donorassay_full', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_in_publication', 'tissue_ontology_term_id'

In [6]:
with h5py.File(file_path, "r") as f:
    # Look at obs and var to get dimensions
    print("\nContents of obs['broad_cell_class'] group:", len(f["obs"]["cell_type"]["categories"]))


Contents of obs['broad_cell_class'] group: 180


## Load 100K cells 

We load a large subset of the data to make sure that we have enough cells in the smaller classes to train on.  We can use the whole dataset
to select cells of each type from.

In [7]:
import h5py
import anndata as ad
from scipy import sparse
import pandas as pd
from src.utils import AnnDataChunker

chunker = AnnDataChunker(file_path, ["cell_type", "broad_cell_class", "donor_id"])
chunker.open()


### Count the number of genes and cells in the dataset

* `f['var']['feature_name']` contains the gene names
* `f['obs']` contains metadata about the cells
* `f['X']['indptr']` contains the indices of the cells in the sparse matrix
* `f['X']['indices']` contains the indices of the genes in the sparse matrix rows
* `f['X']['data']` contains the data for the matrix

## Load the GenePT embedding data
These embeddings were loaded from huggingface in the setup above.

In [8]:
gene_names = chunker.var.feature_name
ensembl_ids = chunker.var.ensembl_id
major_ensembl_ids = pd.Series(ensembl_id.split(".")[0] for ensembl_id in ensembl_ids)

In [9]:
from datasets import load_dataset

gene_info_table_dataset = load_dataset(
    "honicky/genept-composable-embeddings-source-data", "gene_info"
)
gene_info_table = gene_info_table_dataset["train"].to_pandas()
gene_info_table.head()

/Users/rj/personal/GenePT-tools/venv3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since honicky/genept-composable-embeddings-source-data couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'gene_info' at /Users/rj/.cache/huggingface/datasets/honicky___genept-composable-embeddings-source-data/gene_info/0.0.0/5a0eee4f5ea012c2de528b150376dc0d4b3c9e45 (last modified on Tue Jan 14 05:59:50 2025).


,gene_name,ensembl_id,gene_type
0,TSPAN6,ENSG00000000003,protein_coding
1,TNMD,ENSG00000000005,protein_coding
2,DPM1,ENSG00000000419,protein_coding
3,SCYL3,ENSG00000000457,protein_coding
4,C1orf112,ENSG00000000460,protein_coding


In [10]:
huggingface_model_dir = data_dir / "huggingface_model"
gene_embedding = pd.read_parquet(
    huggingface_model_dir
    / "embedding_associations_cell_type_tissue_drug_pathway_openai_large.parquet"
)

# Embed using the mean for duplicate Ensembl IDs

Some of the embeddings have duplicate Ensembl IDs (but separate gene names). Take the mean of the duplicates as a rough heuristic.

In [11]:
gene_embeddings_with_ensembl_id = gene_embedding.merge(
    gene_info_table, left_index=True, right_on="gene_name"
)

In [12]:
gene_embeddings_with_ensembl_id[
    gene_embeddings_with_ensembl_id.ensembl_id == "ENSG00000222005"
]

,0,1,2,3,4,5,6,7,8,9,...,3065,3066,3067,3068,3069,3070,3071,gene_name,ensembl_id,gene_type
83316,-0.008708,0.019564,-0.004104,0.007363,0.001401,-0.015853,0.031096,-0.013775,0.029875,0.012154,...,-0.027197,-0.001720,-0.017885,0.026281,-0.014139,-0.013329,-0.027221,LINC01118,ENSG00000222005,None
83105,-0.018096,0.014083,-0.011138,-0.001666,0.012291,-0.000067,0.038749,-0.007913,0.007358,0.008200,...,-0.016436,-0.012757,-0.025251,0.022767,-0.019888,-0.012172,-0.025777,LINC01119,ENSG00000222005,None


In [13]:
import numpy as np

matching_ensembl_ids = pd.DataFrame(major_ensembl_ids, columns=["ensembl_id"]).merge(
    gene_embeddings_with_ensembl_id, left_on="ensembl_id", right_on="ensembl_id"
)[["gene_name", "ensembl_id"]]
ensembl_id_counts = matching_ensembl_ids.ensembl_id.value_counts()
ensembl_id_counts[ensembl_id_counts > 1]

ensembl_id
ENSG00000000003    2
ENSG00000243485    2
ENSG00000222005    2
ENSG00000264405    2
ENSG00000204792    2
ENSG00000201388    2
ENSG00000222345    2
ENSG00000276234    2
ENSG00000187838    2
ENSG00000202377    2
ENSG00000226364    2
ENSG00000269433    2
ENSG00000183598    2
ENSG00000267151    2
ENSG00000236790    2
ENSG00000264073    2
ENSG00000265134    2
ENSG00000238936    2
ENSG00000270722    2
ENSG00000226444    2
ENSG00000269099    2
ENSG00000204397    2
ENSG00000249532    2
ENSG00000206603    2
ENSG00000284917    2
ENSG00000268942    2
ENSG00000206903    2
ENSG00000063587    2
ENSG00000274020    2
ENSG00000255154    2
ENSG00000090857    2
ENSG00000254508    2
ENSG00000206897    2
ENSG00000251866    2
ENSG00000226419    2
ENSG00000264448    2
ENSG00000206785    2
ENSG00000197927    2
ENSG00000245080    2
ENSG00000269586    2
ENSG00000227518    2
ENSG00000145491    2
ENSG00000221164    2
ENSG00000250331    2
ENSG00000223770    2
ENSG00000269955    2
ENSG00000207187    2
EN

In [14]:
import numpy as np

# Get embeddings without metadata columns
embedding_cols = [
    col
    for col in gene_embeddings_with_ensembl_id.columns
    if col not in ["gene_name", "ensembl_id", "gene_type"]
]

# Group by ensembl_id and take mean of embeddings
merged_embeddings = (
    gene_embeddings_with_ensembl_id.groupby("ensembl_id")[embedding_cols]
    .mean()
    .reset_index()
)

# Renormalize the embeddings
embedding_values = merged_embeddings[embedding_cols].values
norms = np.linalg.norm(embedding_values, axis=1, keepdims=True)
merged_embeddings[embedding_cols] = embedding_values / norms

# Verify the results
print(f"Original shape: {gene_embeddings_with_ensembl_id.shape}")
print(f"After merging duplicates: {merged_embeddings.shape}")

# Verify all duplicates are resolved
duplicate_check = merged_embeddings.ensembl_id.value_counts()
print("\nNumber of remaining duplicates:", (duplicate_check > 1).sum())

Original shape: (37220, 3075)
After merging duplicates: (36573, 3073)

Number of remaining duplicates: 0


In [15]:
merged_embeddings.ensembl_id

0        ENSG00000000003
1        ENSG00000000005
2        ENSG00000000419
3        ENSG00000000457
4        ENSG00000000460
              ...       
36568    ENSGR0000230542
36569    ENSGR0000234958
36570    ENSGR0000236871
36571    ENSGR0000237040
36572    ENSGR0000265658
Name: ensembl_id, Length: 36573, dtype: object

In [16]:
import src.inference as inference

embedding_matrix, valid_indices = inference.create_embedding_matrix_torch(
    merged_embeddings, major_ensembl_ids
)

In [17]:
embedding_matrix.shape

torch.Size([3072, 33258])

In [21]:
from threading import Thread
from queue import Queue
import numpy as np
import time
import torch


adata_filtered = chunker.load_subset(0, 10000, valid_indices)

start_time = time.time()

import sys
import numpy as np

# Check memory addresses and ref counts before
print(f"Original data memory address: {adata_filtered.X.data.data.tobytes()[:10].hex()}")
print(f"Reference count before: {sys.getrefcount(adata_filtered.X.data)}")

# Create tensor
chunk_data_tensor = torch.sparse_csr_tensor(
    torch.from_numpy(adata_filtered.X.indptr),
    torch.from_numpy(adata_filtered.X.indices),
    torch.from_numpy(adata_filtered.X.data),
    size=adata_filtered.X.shape
)

# Check tensor's data memory address
print(f"Tensor data memory address: {chunk_data_tensor.values().data_ptr():x}")
print(f"Reference count after: {sys.getrefcount(adata_filtered.X.data)}")

# Check if data is contiguous and shares memory
print(f"Is tensor contiguous? {chunk_data_tensor.values().is_contiguous()}")
print(f"Original data type: {adata_filtered.X.data.dtype}")
print(f"Tensor data type: {chunk_data_tensor.dtype}")
# Create embeddings for this chunk
cell_embeddings_pt = inference.create_cell_embeddings_torch(
    chunk_data_tensor, embedding_matrix
)

# If tensor is already on CPU, use in-place operation
if cell_embeddings_pt.device.type == 'cpu':
    cell_embeddings = cell_embeddings_pt.numpy()
else:
    # Move to CPU and convert to numpy
    cell_embeddings = cell_embeddings_pt.to("cpu").numpy()

end_time = time.time()

# Calculate metrics
total_time = end_time - start_time
cells_per_second = cell_embeddings.shape[0] / total_time

print(f"Shape of cell embeddings: {cell_embeddings.shape}")
print(f"Total time: {total_time:.2f} seconds")
print(f"Processing speed: {cells_per_second:.1f} cells/second")

/Users/rj/personal/GenePT-tools/venv3.10/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Original data memory address: fd627f3cb9bd4840fd62
Reference count before: 2
Tensor data memory address: 31deb0000
Reference count after: 3
Is tensor contiguous? True
Original data type: float32
Tensor data type: torch.float32
Shape of cell embeddings: (10000, 3072)
Total time: 119.89 seconds
Processing speed: 83.4 cells/second


In [19]:
# import time
# import numpy as np

# # Time the embedding creation
# start_time = time.time()
# cell_embeddings = inference.create_cell_embeddings(
#     adata_filtered.X, embedding_matrix, valid_indices
# )
# end_time = time.time()

# # Calculate metrics
# total_time = end_time - start_time
# cells_per_second = cell_embeddings.shape[0] / total_time

# print(f"Shape of cell embeddings: {cell_embeddings.shape}")
# print(f"Number of genes used: {len(valid_indices)} out of {len(major_ensembl_ids)} total genes")
# print(f"Total time: {total_time:.2f} seconds")
# print(f"Processing speed: {cells_per_second:.1f} cells/second")

Shape of cell embeddings: (100000, 3072)
Number of genes used: 33258 out of 61759 total genes
Total time: 246.57 seconds
Processing speed: 405.6 cells/second


In [ ]:
adata_filtered.obs

,cell_type,broad_cell_class,donor_id
0,"naive thymus-derived CD4-positive, alpha-beta ...",t cell,TSP2
1,B cell,lymphocyte of b lineage,TSP2
2,B cell,lymphocyte of b lineage,TSP2
3,B cell,lymphocyte of b lineage,TSP2
4,"CD8-positive, alpha-beta T cell",t cell,TSP2
...,...,...,...
9995,macrophage,myeloid leukocyte,TSP14
9996,T cell,t cell,TSP14
9997,macrophage,myeloid leukocyte,TSP14
9998,mast cell,granulocyte,TSP14


In [25]:
# def load_cell_metadata(file_path, start_row=0, n_rows=1000, columns=None):
#     """
#     Load metadata for specific cells.

#     Args:
#         file_path: Path to h5ad file
#         start_row: Starting row index
#         n_rows: Number of rows to load
#         columns: List of metadata columns to load (if None, load all)

#     Returns:
#         Dictionary of metadata arrays
#     """
#     with h5py.File(file_path, 'r') as f:
#         metadata = {}
#         obs_keys = list(f['obs'].keys()) if columns is None else columns
#         for key in obs_keys:
#             if key in f['obs']:
#                 column_group = f['obs'][key]
#                 if isinstance(column_group, h5py.Dataset):
#                     metadata[key] = column_group[start_row:start_row + n_rows]
#                 else:
#                     if 'categories' in column_group and 'codes' in column_group:
#                         # Get categories and decode from bytes to strings
#                         categories = [cat.decode('utf-8') for cat in column_group['categories'][:]]
#                         codes = column_group['codes'][start_row:start_row + n_rows]
#                         metadata[key] = np.array([categories[code] for code in codes])
#     return metadata

# # Example usage:
# metadata = load_cell_metadata(file_path, start_row=0, n_rows=1136219,
#                             columns=['cell_type', 'total_counts', 'broad_cell_class', 'donor_id'])
metadata = adata_filtered.obs

print("Loaded metadata keys:", list(metadata.keys()))

# Print first few values of each column
for key in metadata:
    print(f"\nFirst 5 values of {key}:")
    print(metadata[key][:5])

Loaded metadata keys: ['cell_type', 'broad_cell_class', 'donor_id']

First 5 values of cell_type:
0    naive thymus-derived CD4-positive, alpha-beta ...
1                                               B cell
2                                               B cell
3                                               B cell
4                      CD8-positive, alpha-beta T cell
Name: cell_type, dtype: category
Categories (180, object): ['male germ cell', 'spermatocyte', 'spermatid', 'spermatogonium', ..., 'alveolar adventitial fibroblast', 'BEST4+ intestinal epithelial cell, human', 'enteroglial cell', 'unknown']

First 5 values of broad_cell_class:
0                     t cell
1    lymphocyte of b lineage
2    lymphocyte of b lineage
3    lymphocyte of b lineage
4                     t cell
Name: broad_cell_class, dtype: category
Categories (40, object): ['adventitial cell', 'cardiac endothelial cell', 'ciliated epithelial cell', 'conjunctival epithelial cell', ..., 'stromal cell', 't cell', 

In [26]:
embed_genept_pdf = pd.DataFrame(cell_embeddings, index=metadata.index).merge(
    metadata, left_index=True, right_index=True
)
embed_genept_pdf.columns = [str(col) for col in embed_genept_pdf.columns]
embed_genept_pdf

,0,1,2,3,4,5,6,7,8,9,...,3065,3066,3067,3068,3069,3070,3071,cell_type,broad_cell_class,donor_id
0,-0.007720,0.013851,-0.005736,0.010341,0.000320,-0.023683,0.041729,-0.033866,0.035611,0.005238,...,-0.015777,-0.000016,-0.017384,0.023981,-0.007408,-0.003723,-0.018266,"naive thymus-derived CD4-positive, alpha-beta ...",t cell,TSP2
1,-0.005579,0.015617,-0.005555,0.009382,0.001121,-0.022421,0.040697,-0.034355,0.035864,0.004520,...,-0.014589,0.000164,-0.017984,0.023778,-0.007307,-0.004185,-0.018247,B cell,lymphocyte of b lineage,TSP2
2,-0.007412,0.014734,-0.005714,0.010164,0.001523,-0.023222,0.042165,-0.034563,0.036910,0.005400,...,-0.014859,0.000339,-0.017272,0.024080,-0.006957,-0.004390,-0.017801,B cell,lymphocyte of b lineage,TSP2
3,-0.008063,0.014125,-0.005429,0.010397,-0.000308,-0.023580,0.041392,-0.034021,0.035953,0.005418,...,-0.015378,-0.000165,-0.017942,0.023627,-0.007249,-0.003548,-0.018380,B cell,lymphocyte of b lineage,TSP2
4,-0.006627,0.014602,-0.005593,0.009993,0.002419,-0.021874,0.041174,-0.034943,0.036720,0.005483,...,-0.015101,0.000418,-0.018152,0.023877,-0.006763,-0.004058,-0.017972,"CD8-positive, alpha-beta T cell",t cell,TSP2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.007187,0.015967,-0.005466,0.009160,0.004823,-0.021902,0.038536,-0.036308,0.038071,0.003966,...,-0.014213,0.001092,-0.017817,0.023512,-0.007067,-0.004614,-0.018447,macrophage,myeloid leukocyte,TSP14
9996,-0.007834,0.014847,-0.005910,0.009447,0.001457,-0.021850,0.040369,-0.034805,0.038106,0.005192,...,-0.014837,0.000930,-0.017083,0.023103,-0.006398,-0.003092,-0.017232,T cell,t cell,TSP14
9997,-0.010408,0.017290,-0.005335,0.008223,0.005261,-0.021961,0.035685,-0.036731,0.037943,0.001932,...,-0.014933,0.000962,-0.018446,0.023422,-0.007367,-0.004373,-0.018508,macrophage,myeloid leukocyte,TSP14
9998,-0.006572,0.015467,-0.005140,0.008197,0.000863,-0.023032,0.040971,-0.035768,0.036060,0.004490,...,-0.014846,0.000088,-0.017184,0.023590,-0.007205,-0.004473,-0.017510,mast cell,granulocyte,TSP14


In [27]:
embed_genept_pdf.to_parquet(data_dir / "tabula_sapiens_10k_genept_embedding_cell_type_tissue_drug_pathway_openai_large.parquet")

In [28]:
cell_type_labels = pd.Categorical(
    pd.Series(metadata["broad_cell_class"])[: len(cell_embeddings)]
)
cell_type_labels

['t cell', 'lymphocyte of b lineage', 'lymphocyte of b lineage', 'lymphocyte of b lineage', 't cell', ..., 'myeloid leukocyte', 't cell', 'myeloid leukocyte', 'granulocyte', 'granulocyte']
Length: 10000
Categories (40, object): ['adventitial cell', 'cardiac endothelial cell', 'ciliated epithelial cell', 'conjunctival epithelial cell', ..., 'stromal cell', 't cell', 'transitional epithelial cell', 'vestibular dark cell']

In [29]:
metadata["broad_cell_class"].value_counts()

broad_cell_class
t cell                             1299
lymphocyte of b lineage            1217
contractile cell                   1077
granulocyte                         962
myeloid leukocyte                   931
stem cell                           711
endothelial cell                    654
stromal cell                        593
fibroblast                          519
innate lymphoid cell                474
epithelial cell                     468
endo-epithelial cell                242
intestinal epithelial cell          222
glandular epithelial cell           118
hematopoietic cell                  114
adventitial cell                     85
dendritic cell                       79
epithelial cell of lung              61
cardiac endothelial cell             39
erythroid lineage cell               31
ciliated epithelial cell             18
duct epithelial cell                 18
secretory cell                       16
stratified epithelial cell           14
meso-epithelial cell   

In [30]:
donor_ids = pd.Categorical(metadata["donor_id"][: len(cell_embeddings)])
donor_ids

['TSP2', 'TSP2', 'TSP2', 'TSP2', 'TSP2', ..., 'TSP14', 'TSP14', 'TSP14', 'TSP14', 'TSP14']
Length: 10000
Categories (24, object): ['TSP1', 'TSP2', 'TSP3', 'TSP4', ..., 'TSP26', 'TSP27', 'TSP28', 'TSP30']

In [31]:
cell_embeddings_pdf = pd.DataFrame(
    cell_embeddings, columns=list(range(cell_embeddings.shape[1]))
)
cell_embeddings_pdf["cell_type"] = cell_type_labels
cell_embeddings_pdf["donor_id"] = donor_ids

In [32]:
cell_embeddings_pdf.shape

(10000, 3074)

In [33]:
from sklearn.decomposition import PCA
import umap
import matplotlib.pyplot as plt
import plotly.express as px

# Convert all column names to strings before applying PCA
# features_24_weeks_df.columns = features_24_weeks_df.columns.astype(str)

# Now run PCA
# pca = PCA(n_components=50)
# pca_embeddings = pca.fit_transform(features_24_weeks_df.drop(columns=['drug dose', 'external_id', 'ifn status', "subject sex", "subject age", "subject_id"]))
# Then apply UMAP to the PCA results
reducer = umap.UMAP(random_state=42)
umap_sample_pdf = cell_embeddings_pdf.sample(2000)
umap_embeddings = reducer.fit_transform(
    umap_sample_pdf.drop(columns=["cell_type", "donor_id"])
)

# Create a DataFrame with the UMAP coordinates using the original DataFrame's index
umap_df = pd.DataFrame(
    umap_embeddings, columns=["UMAP1", "UMAP2"], index=umap_sample_pdf.index
)  # Use the original DataFrame's index

# Merge with sample attributes to get metadata for coloring
umap_df = umap_df.merge(
    umap_sample_pdf[["cell_type", "donor_id"]], left_index=True, right_index=True
)
# Create the plot
fig = px.scatter(
    umap_df,
    x="UMAP1",
    y="UMAP2",
    color="cell_type",
    opacity=0.7,
    title="UMAP Visualization of Gene Expression Embeddings",
)

# Update layout
fig.update_layout(title={"y": 0.95, "x": 0.5, "xanchor": "center", "yanchor": "top"})

fig.show()

/Users/rj/personal/GenePT-tools/venv3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/rj/personal/GenePT-tools/venv3.10/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
